
# Q-Learning (Tabular) — Discrete Environment

**Goal:** Implement classic *tabular* Q-Learning on a simple discrete Gymnasium environment (e.g., FrozenLake).  
We will build the Q-table, use ε-greedy exploration, apply the Bellman update, and evaluate the learned policy.

**Key ideas:**  
- Q-table `Q[s, a]` stores value estimates  
- ε-greedy balances exploration vs exploitation  
- Bellman update nudges Q toward `r + γ max_a' Q(s', a')`  
- Works only for *small discrete* state spaces



### Why this step?  
We install the required packages so the notebook can run anywhere (local, Colab).  
- **gymnasium**: modern Gym API for RL environments  
- **stable-baselines3**: popular, well-maintained RL algorithms (DQN, PPO, A2C)  
- **tensorboard** (optional): view training curves  
- **matplotlib**: simple plotting

### How it works  
`pip` installs packages into the active environment. In Colab, this writes to the session; locally, it writes to your venv.


In [ ]:

# If running locally and already installed, you can skip this.
# In Colab, keep this cell.
# Note: remove the `-q` if you want to see full logs.
!pip install -q gymnasium[classic-control]==0.29.1 stable-baselines3==2.3.2 tensorboard matplotlib numpy



### Why this step?
Import dependencies and create a **discrete** environment so that we can index states/actions in a Q-table.

### How it works
- `gymnasium.make()` constructs the environment.  
- We pick `FrozenLake-v1` (deterministic mode) to make convergence easier to observe.


In [ ]:

import numpy as np
import gymnasium as gym

env = gym.make("FrozenLake-v1", is_slippery=False)
n_states = env.observation_space.n
n_actions = env.action_space.n

n_states, n_actions



### Why this step?
Initialize the **Q-table** and hyperparameters that govern learning speed, discounting, exploration, and training duration.

### How it works
- `Q` starts at zeros.  
- `alpha` (learning rate) controls how aggressively we update.  
- `gamma` discounts future rewards.  
- `epsilon` decays each episode to reduce exploration over time.


In [ ]:

Q = np.zeros((n_states, n_actions))

alpha = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995

n_episodes = 3000
max_steps = 200

returns = []  # track episode rewards



### Why this step?
Define ε-greedy **action selection** so we sometimes try random actions (exploration) rather than always choosing the current best (exploitation).

### How it works
If `rand < ε`: random action. Else: `argmax(Q[state])`.


In [ ]:

def epsilon_greedy_action(Q, state, epsilon, action_space):
    if np.random.rand() < epsilon:
        return action_space.sample()
    return int(np.argmax(Q[state]))



### Why this step?
Run the **training loop** across episodes and steps, applying the Q-Learning update.

### How it works
For each transition `(s, a, r, s')`, update:
```
Q[s,a] ← Q[s,a] + α [ r + γ * max_a' Q[s', a'] − Q[s,a] ]
```
We also decay `epsilon` each episode.


In [ ]:

for ep in range(n_episodes):
    state, _ = env.reset(seed=ep)
    done = False
    ep_return = 0.0

    for t in range(max_steps):
        action = epsilon_greedy_action(Q, state, epsilon, env.action_space)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Bellman target and update
        best_next = np.max(Q[next_state])
        td_target = reward + gamma * best_next
        td_error = td_target - Q[state, action]
        Q[state, action] += alpha * td_error

        state = next_state
        ep_return += reward
        if done:
            break

    returns.append(ep_return)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

len(returns), returns[-10:]



### Why this step?
Extract a **deterministic policy** from the learned `Q` and evaluate it without exploration.

### How it works
- Policy: `π(s) = argmax_a Q[s, a]`  
- Run several evaluation episodes and compute mean reward.


In [ ]:

policy = np.argmax(Q, axis=1)
print("Policy (state → action):", policy)

def evaluate(policy, env, n_eval_episodes=100):
    total = 0.0
    for i in range(n_eval_episodes):
        s, _ = env.reset(seed=10_000 + i)
        done = False
        while not done:
            a = int(policy[s])
            s, r, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            total += r
    return total / n_eval_episodes

avg_ret = evaluate(policy, env, n_eval_episodes=200)
print(f"Average return over 200 eval episodes: {avg_ret:.3f}")



### Why this step?
(Optional) **Plot** the training returns to visualize learning progress.

### How it works
We use `matplotlib` to draw the episode reward trajectory and a rolling mean.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.plot(returns, label="Episode return")
window = 50
if len(returns) >= window:
    mov = np.convolve(returns, np.ones(window)/window, mode='valid')
    plt.plot(range(window-1, window-1+len(mov)), mov, label=f"Moving avg ({window})")
plt.xlabel("Episode")
plt.ylabel("Return")
plt.legend()
plt.title("Q-Learning on FrozenLake (deterministic)")
plt.show()
